## load packages to be used

In [1]:
import planetary_computer
import pystac_client # connecting to the STAC API
from pystac.extensions.eo import EOExtension as eo
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt
import rasterio
from rasterio import windows
from rasterio import features
from rasterio import warp
import dask.array as da # handle dask arrays
import geopandas as gpd # handle geospatial data frames
from IPython.display import Image # visualize URLs
from rasterio.enums import Resampling # perform re-sampling operations
import shapely # create vector objects
import stackstac # build an on-demand STAC data cube
import numpy as np
from PIL import Image
import warnings
warnings.simplefilter("ignore", category=UserWarning)
print("Packages loaded successfully")

Packages loaded successfully


In [2]:
# Connect to Planetary Computer STAC API
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

aoi_bbox = [33.9, 0.1, 34.6, 0.8]
# Convert to a proper GeoJSON Polygon
area_of_interest = {
    "type": "Polygon",
    "coordinates": [[
        [aoi_bbox[0], aoi_bbox[1]],  # Bottom-left
        [aoi_bbox[0], aoi_bbox[3]],  # Top-left
        [aoi_bbox[2], aoi_bbox[3]],  # Top-right
        [aoi_bbox[2], aoi_bbox[1]],  # Bottom-right
        [aoi_bbox[0], aoi_bbox[1]],  # Close the loop
    ]]
}

time_of_interest = "2024-01-01/2024-12-31"

# Define the search parameters for Busia County
search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects=area_of_interest,
    datetime=time_of_interest,
    query={"eo:cloud_cover": {"lt": 10}},
)

# Check how many items were returned
items = search.item_collection()
print(f"Found {len(items)} Sentinel-2 scenes for Busia County")

Found 63 Sentinel-2 scenes for Busia County


To explore the items as data frames we can either convert to a geopandas table in Python or to an sf table in R. We added an extra fid column to allow index matching to the original item list.

In [3]:
items_df = gpd.GeoDataFrame.from_features(items.to_dict(), crs="epsg:4326")
items_df

,geometry,datetime,platform,instruments,s2:mgrs_tile,constellation,s2:granule_id,eo:cloud_cover,s2:datatake_id,s2:product_uri,...,s2:nodata_pixel_percentage,s2:unclassified_percentage,s2:not_vegetated_percentage,s2:degraded_msi_data_percentage,s2:high_proba_clouds_percentage,s2:reflectance_conversion_factor,s2:medium_proba_clouds_percentage,s2:saturated_defective_pixel_percentage,proj:code,s2:dark_features_percentage
0,"POLYGON ((33.6331 -0.08847, 33.64531 -0.03302,...",2024-12-31T07:53:31.024000Z,Sentinel-2A,[msi],36NWF,Sentinel 2,S2A_OPER_MSI_L2A_TL_2APS_20241231T110447_A0497...,2.682315,GS2A_20241231T075331_049753_N05.11,S2A_MSIL2A_20241231T075331_N0511_R135_T36NWF_2...,...,75.250989,0.374750,1.871524,0.0447,0.000402,1.034080,0.009170,0.0,EPSG:32636,NaN
1,"POLYGON ((34.03295 0.90475, 34.01381 0.81771, ...",2024-12-29T08:02:39.024000Z,Sentinel-2B,[msi],36NXF,Sentinel 2,S2B_OPER_MSI_L2A_TL_2BPS_20241229T100143_A0408...,0.045219,GS2B_20241229T080239_040816_N05.11,S2B_MSIL2A_20241229T080239_N0511_R035_T36NXF_2...,...,95.964497,0.901336,3.399887,0.0000,0.000000,1.033919,0.000164,0.0,EPSG:32636,NaN
2,"POLYGON ((33.98661 0.69373, 33.98108 0.66853, ...",2024-12-29T08:02:39.024000Z,Sentinel-2B,[msi],36NWF,Sentinel 2,S2B_OPER_MSI_L2A_TL_2BPS_20241229T100143_A0408...,2.268725,GS2B_20241229T080239_040816_N05.11,S2B_MSIL2A_20241229T080239_N0511_R035_T36NWF_2...,...,7.052120,0.496113,2.978773,0.0209,0.141573,1.033919,0.499365,0.0,EPSG:32636,NaN
3,"POLYGON ((33.89875 0.9048, 34.88531 0.90442, 3...",2024-12-11T07:53:31.024000Z,Sentinel-2A,[msi],36NXF,Sentinel 2,S2A_OPER_MSI_L2A_TL_2APS_20241211T111851_A0494...,3.419567,GS2A_20241211T075331_049467_N05.11,S2A_MSIL2A_20241211T075331_N0511_R135_T36NXF_2...,...,0.000000,0.194571,3.354179,0.0089,1.792682,1.030660,1.617317,0.0,EPSG:32636,NaN
4,"POLYGON ((33.63241 -0.08847, 33.65559 0.01676,...",2024-12-11T07:53:31.024000Z,Sentinel-2A,[msi],36NWF,Sentinel 2,S2A_OPER_MSI_L2A_TL_2APS_20241211T111851_A0494...,0.761265,GS2A_20241211T075331_049467_N05.11,S2A_MSIL2A_20241211T075331_N0511_R135_T36NWF_2...,...,75.182420,0.432283,2.006057,0.0445,0.230025,1.030660,0.440238,0.0,EPSG:32636,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,"POLYGON ((33.6324 -0.08847, 33.64342 -0.03849,...",2024-02-20T07:49:49.024000Z,Sentinel-2B,[msi],36NWF,Sentinel 2,S2B_OPER_MSI_L2A_TL_MSFT_20240220T115252_A0363...,4.565432,GS2B_20240220T074949_036340_N05.10,S2B_MSIL2A_20240220T074949_N0510_R135_T36NWF_2...,...,75.549877,0.772639,4.930963,0.0000,1.566148,1.024975,1.821803,0.0,EPSG:32636,0.054171
59,"POLYGON ((34.03246 0.90475, 34.01997 0.84814, ...",2024-02-13T08:00:09.024000Z,Sentinel-2B,[msi],36NXF,Sentinel 2,S2B_OPER_MSI_L2A_TL_MSFT_20240213T115120_A0362...,8.149595,GS2B_20240213T080009_036240_N05.10,S2B_MSIL2A_20240213T080009_N0510_R035_T36NXF_2...,...,96.003449,1.068437,7.112040,0.0000,2.360441,1.027631,5.789153,0.0,EPSG:32636,0.002740
60,"POLYGON ((34.03837 0.90475, 34.03401 0.88494, ...",2024-01-24T08:01:29.024000Z,Sentinel-2B,[msi],36NXF,Sentinel 2,S2B_OPER_MSI_L2A_TL_MSFT_20240124T115914_A0359...,6.281681,GS2B_20240124T080129_035954_N05.10,S2B_MSIL2A_20240124T080129_N0510_R035_T36NXF_2...,...,95.636785,1.096893,7.715816,0.0000,1.983837,1.032908,4.297844,0.0,EPSG:32636,0.011102
61,"POLYGON ((34.05493 0.90474, 34.03778 0.82666, ...",2024-01-19T08:02:41.024000Z,Sentinel-2A,[msi],36NXF,Sentinel 2,S2A_OPER_MSI_L2A_TL_MSFT_20240119T124049_A0447...,0.000000,GS2A_20240119T080241_044791_N05.10,S2A_MSIL2A_20240119T080241_N0510_R035_T36NXF_2...,...,94.353914,1.047107,7.729396,0.0000,0.000000,1.033630,0.000000,0.0,EPSG:32636,0.002351


When we have the items as data frame, we can further filter the data based on the table columns. Below, we filter for data with a cloud cover lower than 2% and with a no-data pixel percentage below 10%.

To illustrate how to fetch item properties, we can select the first item in our list and get the datetime of this scene.

In [4]:
ids = items_df.loc[
  (items_df['eo:cloud_cover'] <= 2) &
  (items_df['s2:nodata_pixel_percentage'] <= 10)
]
item = items[ids.index[0]]
item.datetime

datetime.datetime(2024, 8, 28, 7, 46, 9, 24000, tzinfo=tzutc())

In [5]:
print(f"Number of STAC items: {len(items)}")

Number of STAC items: 63


In [6]:
import itertools

# Extract all available asset names from STAC items
available_assets = set(
    itertools.chain.from_iterable(
        item.assets.keys() for item in items
    )
)

print("Available Assets:", available_assets)

Available Assets: {'B09', 'B01', 'B08', 'rendered_preview', 'SCL', 'datastrip-metadata', 'B07', 'granule-metadata', 'WVP', 'safe-manifest', 'B05', 'B06', 'B12', 'B04', 'inspire-metadata', 'B02', 'tilejson', 'AOT', 'visual', 'B11', 'B03', 'B8A', 'product-metadata', 'preview'}


+ "B02" → Blue
+ "B03" → Green
+ "B04" → Red
+ "B08" → Near Infrared (NIR)
+ "SCL" → Scene Classification Layer

In [7]:
import stackstac

# Stack the data
data_cube = stackstac.stack(
    items,
    assets=["B08", "B04"],  # Select NIR and Red bands
    resolution=10,  # Sentinel-2 native resolution in meters
    epsg=32636,  # EPSG of Busia
    chunksize=2048,  # Optimize chunk size for Dask
)

data_cube # Check if (time, y, x) structure is correct

<xarray.DataArray 'stackstac-8b0c2b9b9f50c198316eba3bfd8ccdf6' (time: 63,
                                                                band: 2,
                                                                y: 10986,
                                                                x: 20986)> Size: 232GB
dask.array<fetch_raster_window, shape=(63, 2, 10986, 20986), dtype=float64, chunksize=(1, 1, 2048, 2048), chunktype=numpy.ndarray>
Coordinates: (12/44)
  * time                                     (time) datetime64[ns] 504B 2024-...
    id                                       (time) <U54 14kB 'S2A_MSIL2A_202...
  * band                                     (band) <U3 24B 'B08' 'B04'
  * x                                        (x) float64 168kB 5e+05 ... 7.09...
  * y                                        (y) float64 88kB 1.001e+05 ... -...
    sat:orbit_state                          <U10 40B 'descending'
    ...                                       ...
    title                                    (band) <U18 144B 'Band 8 - NIR -...
    gsd                                      float64 8B 10.0
    common_name                              (band) <U3 24B 'nir' 'red'
    center_wavelength                        (band) float64 16B 0.842 0.665
    full_width_half_max                      (band) float64 16B 0.145 0.038
    epsg                                     int64 8B 32636
Attributes:
    spec:        RasterSpec(epsg=32636, bounds=(499970, -9790, 709830, 100070...
    crs:         epsg:32636
    transform:   | 10.00, 0.00, 499970.00|\n| 0.00,-10.00, 100070.00|\n| 0.00...
    resolution:  10

In [8]:
import xarray as xr
import numpy as np

# Select NIR (B08) and Red (B04)
nir = data_cube.sel(band="B08")
red = data_cube.sel(band="B04")

# Mask invalid values
valid_mask = (nir > 0) & (red > 0)

# Compute NDVI using Dask (lazy evaluation)
ndvi = xr.where(valid_mask, (nir - red) / (nir + red), np.nan)

In [10]:
subset = ndvi.isel(time=0, x=slice(0, 500), y=slice(0, 500))
print(subset)

<xarray.DataArray 'stackstac-8b0c2b9b9f50c198316eba3bfd8ccdf6' (y: 500, x: 500)> Size: 2MB
dask.array<getitem, shape=(500, 500), dtype=float64, chunksize=(500, 500), chunktype=numpy.ndarray>
Coordinates: (12/39)
    time                                     datetime64[ns] 8B 2024-01-19T08:...
    id                                       <U54 216B 'S2A_MSIL2A_20240119T0...
  * x                                        (x) float64 4kB 5e+05 ... 5.05e+05
  * y                                        (y) float64 4kB 1.001e+05 ... 9....
    sat:orbit_state                          <U10 40B 'descending'
    constellation                            <U10 40B 'Sentinel 2'
    ...                                       ...
    s2:generation_time                       <U27 108B '2024-01-19T12:40:49.2...
    s2:vegetation_percentage                 float64 8B 91.22
    s2:unclassified_percentage               float64 8B 1.047
    proj:shape                               object 8B {10980}
    gsd      

In [11]:
subset = ndvi.isel(time=0, x=slice(0, 500), y=slice(0, 500)).compute()
print(subset.values)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [ ]:
# Ensure time is in datetime format
ndvi_monthly = ndvi.assign_coords(time=ndvi.time.dt.month)

# Compute monthly mean NDVI
ndvi_monthly_mean = ndvi_monthly.groupby("time").mean(dim="time").compute()
ndvi_monthly_mean

In [ ]:
import stackstac
import dask.array as da
from rasterio.enums import Resampling

# Define bounding box correctly (min lon, min lat, max lon, max lat)
bbox = (33.9, 0.1, 34.6, 0.8)
assets = ["B02", "B03", "B04", "B08", "SCL"]

# Check if STAC items exist
if not items:
    raise ValueError("No STAC items found for the given bounding box and time range.")

# Stack the STAC items into a datacube
s2_cube_all = stackstac.stack(
    items,
    assets=assets,
    resolution=10,
    bounds_latlon=bbox,  # Ensure this is correct
    resampling=Resampling.bilinear, 
    epsg=3857
)

# Ensure the cube has valid bounds
if s2_cube_all.rio.bounds() is None:
    raise ValueError("Invalid bounds in the stacked datacube.")

# Select cloud mask layer (Scene Classification Layer - SCL)
scl = s2_cube_all.sel(band=["SCL"])

# Define valid SCL values (3 = Cloud Shadow, 8 & 9 = Clouds)
s2_mask = da.isin(scl, [3, 8, 9])

# Apply mask to filter out clouds
s2_cube = s2_cube_all.where(~s2_mask)

# Compute NDVI
nir = s2_cube.sel(band="B08")  # Near Infrared
red = s2_cube.sel(band="B04")  # Red
ndvi_cube = (nir - red) / (nir + red)

print(f"ndvi Datacube shape: {ndvi_cube.shape}")
ndvi_cube

In [ ]:
print("Extracted Bands:", s2_cube_all.band.values)

In [ ]:
"""
from pystac_client import Client

catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = client.search(
    collections=[collection],
    intersects=area_of_interest,
    datetime=time_of_interest,
    query=["eo:cloud_cover<10"],
)

items = list(search.item_collection())
print(f"Found {len(items)} items")

# Print available dates
dates = [item.datetime.strftime("%Y-%m-%d") for item in items]
print("Available Dates:", dates)
"""

In [ ]:
"""# Check available assets for each date
for item in items:
    print(f"Date: {item.datetime}")
    print(f"Available Assets: {list(item.assets.keys())}\n")"""

### Computing Monthly Mean NDVI

In [ ]:
# Computing Monthly Mean NDVI
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

# Ensure time is in datetime format
s2_cube = s2_cube.assign_coords(time=s2_cube.time.dt.strftime("%Y-%m"))  # Format time as 'YYYY-MM'

# Group by month
ndvi_cube_monthly = ndvi_cube.groupby("time").mean(dim="time")  # Compute mean per month

# Count number of scenes per month
scene_counts = ndvi_cube.groupby("time").count(dim="time")

# Print count of scenes for each month
print(scene_counts)

In [ ]:
ndvi_cube_monthly

In [ ]:
# Ensure time is in datetime format
ndvi_cube_monthly = ndvi_cube.assign_coords(time=ndvi_cube.time.dt.month)

# Compute monthly mean NDVI
ndvi_monthly_mean = ndvi_cube_monthly.groupby("time").mean(dim="time")
ndvi_monthly_mean

In [ ]:
# output_nc = "~/geospatial_workflows_automation/geospatial_datacube/data/ndvi_monthly_means.nc"
# ndvi_monthly_mean.to_netcdf(output_nc)
# print(f"NetCDF saved at: {output_nc}")

In [ ]:
# Verify Dimentions and coordinate systems
print(ndvi_monthly_mean.dims)  # Should return {'time': 12, 'y': 7793, 'x': 7793}
print(ndvi_monthly_mean.coords)  # Ensure coordinates like EPSG, x, y, and time are correct

In [ ]:
subset = ndvi_monthly_mean.isel(time=0, x=slice(0, 500), y=slice(0, 500))
print(subset)

In [ ]:
subset = ndvi_monthly_mean.isel(time=0, x=slice(0, 500), y=slice(0, 500)).compute()
print(subset.values)

In [ ]:
print(ndvi_monthly_mean.isnull().sum().values)  # Check how many NaN values exist

In [ ]:
print(ndvi_monthly_mean.count().values)  # Should return a number > 0 if valid data exists

In [ ]:
# import xarray as xr

# file_path = "~/geospatial_workflows_automation/geospatial_datacube/data/ndvi_monthly_means.nc"
# ds = xr.open_dataset(file_path)

# print(ds)  # Prints dataset structure
# print(ds.variables)  # Lists all variables

In [ ]:
# # Example: Extract NDVI at pixel index (x=1000, y=2000) for all months
# ndvi_pixel = ds["stackstac-c9ab6691a0387bfa3e69bb26c79391bb"].isel(x=1000, y=2000)
# print(ndvi_pixel)

In [ ]:
# # Extract NDVI at (x=1000, y=2000) for January (time index 0)
# ndvi_jan = ds["stackstac-c9ab6691a0387bfa3e69bb26c79391bb"].isel(x=1000, y=2000, time=11)
# print(ndvi_jan.values)

In [ ]:
# lon = 36.8219  # Example longitude
# lat = -1.2921  # Example latitude

# ndvi_point = ds["stackstac-c9ab6691a0387bfa3e69bb26c79391bb"].sel(x=lon, y=lat, method="nearest")
# print(ndvi_point.values)